# Цель проекта: Определение стоимости автомобилей / Project Goal: Determining the value of cars.

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

ENG:
The used car sales service "Not beaten, not painted" is developing an application to attract new customers. With it, you can quickly find out the market value of your car. You have historical data at your disposal: technical specifications, configurations, and car prices. You need to build a model to determine the value.

The client is concerned with:

- prediction quality;
- prediction speed;
- training time.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Подготовка данных / Data preparation

### Загружаем библиотеки / Loading libraries



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor


### Загружаем данные и получаем первичную иформацию / We load the data and get the primary information

In [ ]:
df=pd.read_csv('/datasets/autos.csv')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df['Price'].hist();

In [ ]:
sns.heatmap(df[[i for i in df.columns if i!='NumberOfPictures']].corr(method='spearman'));

### Переведем часть признаков в другой формат / Let's translate some of the features into another format

In [ ]:
for i in ['PostalCode','Brand','Model','VehicleType','Gearbox']:
    df[i]=df[i].astype("category")
for i in ['LastSeen','DateCrawled','DateCreated']:
    df[i]=pd.to_datetime(df[i],format='%Y-%m-%d %H:%M:%S')

### Проверим на пропуски и дубликаты / Let's check for omissions and duplicates

In [ ]:
missing_values = df.isna().sum()

missing_columns = missing_values[missing_values > 0]



ax = missing_columns.plot(kind='barh', figsize=(15, 6), grid=True, color='b', alpha=0.6, label='До')

plt.title('Пропущенные значения')
plt.ylabel('Колонки с пропусками')
plt.xlabel('Количество пропусков')
plt.legend()
%store ax

In [ ]:
print('Количество дубликатов',df.duplicated().sum())

### Вывод / Conclusion

По первичным данным которые мы можем увидеть,можно сделать вывод что у нас достаточно крупная выборка,у которой много пропусков,типы данных в большинстве случаев категориальные и численные.Целевая переменная имеет асиметричное распредление.Так же можем увидеть сильную взаимосвязь у несколкьих признаков с целевой переенной это дата регистрации и мощность.Последовательность действий:
* Убирем аномальные значения
* Для востановления пропусков я создам функцию с простой моделькой которая сможет предсказать пропуски и заменить их
* Оставшиеся пропуски заменю рядом стоящими значениями
* Дубликаты из за незначительного количества просто удалю
* После этих действий изменю типы данных у некоторых признаков,часть из них удалю,а часть наоборот преобразую.

ENG: 
Based on the initial data we can see, it can be concluded that we have a fairly large sample, with many missing values. The data types in most cases are categorical and numerical. The target variable has an asymmetric distribution. We can also see a strong correlation of a few features with the target variable, namely the registration date and power. Here's the sequence of actions:

* Remove anomalous values
* To recover missing data, I will create a function with a simple model that can predict and replace missing values
* Remaining missing values will be replaced by neighboring values
* I will simply delete duplicates due to their insignificant quantity
* After these actions, I will change the data types of some features, delete some of them, and transform some others.

##  Обработка пропусков / Missing Data Handling

In [ ]:
df=df.loc[(df['RegistrationYear']<2016) & (df['RegistrationYear']>1910) ]
df=df.loc[df['Price']>10]
df=df.loc[df['Power']>1]

In [ ]:
def missing(data,features,target):# Функция замены пропусков
    
    columns = [i for i in features]
    
    
    new_data = data.loc[(-data[target].isna())&(-data[columns[0]].isna())&(-data[columns[1]].isna())&
                   (-data[columns[2]].isna())&(-data[columns[3]].isna()),features+[target]]

    predict = data.loc[(data[target].isna())&(-data[columns[0]].isna())&(-data[columns[1]].isna())&
                   (-data[columns[2]].isna())&(-data[columns[3]].isna()),features]
    
    
    
    features=new_data.drop(target,axis=1)
    
    target=new_data[target]
    
    label=LabelEncoder()
    
    targets=label.fit_transform(target)
    
    
    encoder=OrdinalEncoder()
    
    new_transform=encoder.fit_transform(np.concatenate((features,predict),axis=0))
    
    features=new_transform[:new_data.shape[0]]
    
    predict=new_transform[new_data.shape[0]:]
    
    param={'max_depth':range(4,20,2)}
    
    
    
    
    grid=RandomizedSearchCV(DecisionTreeClassifier(),param,cv=3)
    grid.fit(features,targets)
    predict=grid.predict(predict)
    print(f'Точность модели при заполнении признака  :{grid.best_score_}')
    return label.inverse_transform(np.round(predict))

### Заменяем пропуски / Replacing Missing Data

In [ ]:
df.loc[(df['Repaired'].isna())&(-df['Gearbox'].isna())&(-df['RegistrationYear'].isna())&
                   (-df['Brand'].isna())&(-df['Kilometer'].isna()),'Repaired']=missing(df,['Gearbox','Brand','Kilometer','RegistrationYear'],'Repaired')

df.loc[(df['Model'].isna())&(-df['Gearbox'].isna())&(-df['RegistrationYear'].isna())&
                   (-df['Brand'].isna())&(-df['VehicleType'].isna()),'Model']=missing(df,['Gearbox','Brand','VehicleType','RegistrationYear'],'Model')

df.loc[(df['Gearbox'].isna())&(-df['Model'].isna())&(-df['RegistrationYear'].isna())&
                   (-df['Brand'].isna())&(-df['VehicleType'].isna()),'Gearbox']=missing(df,['Model','Brand','VehicleType','RegistrationYear'],'Gearbox')

df.loc[(df['VehicleType'].isna())&(-df['Gearbox'].isna())&(-df['RegistrationYear'].isna())&
                   (-df['Brand'].isna())&(-df['Model'].isna()),'VehicleType']=missing(df,['Gearbox','Brand','Model','RegistrationYear'],'VehicleType')

df.loc[(df['FuelType'].isna())&(-df['Gearbox'].isna())&(-df['RegistrationYear'].isna())&
                   (-df['Brand'].isna())&(-df['Model'].isna()),'FuelType']=missing(df,['Gearbox','Brand','Model','RegistrationYear'],'FuelType')

In [ ]:
df['FuelType']=df.groupby(['Brand','RegistrationYear','Model'])['FuelType'].apply(lambda x: x.ffill().bfill())
df['VehicleType']=df.groupby(['Brand','Model','RegistrationYear'])['VehicleType'].apply(lambda x: x.ffill().bfill())
df['Gearbox']=df.groupby(['Brand','Model','RegistrationYear'])['Gearbox'].apply(lambda x: x.ffill().bfill())

In [ ]:
%store -r ax

new_missing_values = df.isna().sum()
new_missing_columns = new_missing_values[new_missing_values > 0]

new_missing_columns.plot(kind='barh', grid=True, color='r', alpha=0.6, ax=ax, label='После')

ax.legend()

plt.show()


In [ ]:
df.shape

### Вывод / Conclusion
Как мы видим количесвто пропсуков значительно сократилось,в связи с чем мы можем оставшиеся удалить. / 
As we can see, the number of missing values has significantly decreased, allowing us to delete the remaining ones.

In [ ]:
df=df.dropna()
df=df.drop_duplicates()

In [ ]:
print(f'Количество пропусков : {df.isna().sum()}')
print()
print(f'Количество lдубликатов : {df.duplicated().sum()}')

## Работа с признаками / Work with features

### Создание новых признаков / Create new features

In [ ]:
df['how_long']=(df['LastSeen']-df['DateCreated']).astype('timedelta64[D]')

df['year_created']=(pd.DatetimeIndex(df['DateCreated']).year).astype("category")

df['month_created']=pd.DatetimeIndex(df['DateCreated']).month.astype("category")

df['Repaired']=np.where(df['Repaired']=='yes',1,0)
for i in ['RegistrationYear','FuelType','Repaired']:
    df[i]=df[i].astype("category")

### Удаление не нужных признаков / Removing unnecessary features

In [ ]:
df=df.drop(['DateCrawled','RegistrationMonth','DateCreated','LastSeen','NumberOfPictures'],axis=1)

In [ ]:
print(df.duplicated().sum())
df=df.drop_duplicates()

### Анализ

In [ ]:
for i in ['Brand','RegistrationYear','VehicleType','how_long','month_created','Repaired']:
    
    df.pivot_table(index=i,values='Price',aggfunc='mean').plot(kind='bar',figsize=(15,5),color='g',grid=True,legend=False)
    plt.xlabel(i)
    plt.ylabel('Средняя цена')
    
    plt.show()


### Вывод / Conclusion 

После небольшого анализа ,мы можем увидеть что некоторые признаки отличаються по цене,конечно большинство из них вполне логичны,но и опять же эти графики дают представление об общей картине.Что ороже стоят машины которые не были биты премиальных брендов,так же можем увидеть что быстрее продаються машины дешовые,так же моно понять что все старые машины это какой то дорогой раритет так как стоимость большая а дата регистрации очень старая,так же можем понять в какие месяца какой тип машин продаеться.

ENG:

After a brief analysis, we can see that some features vary in price, most of which are quite logical. Yet again, these graphs provide an overview of the general picture. More expensive cars are those that haven't been damaged and belong to premium brands. We can also see that cheaper cars sell faster. Additionally, it can be understood that all old cars are some sort of expensive rarity since the cost is high and the registration date is very old. We can also understand in which months certain types of cars are sold.


## Обучение моделей / Learning models

### Выделим признаки и целевой признак / Let's highlight the signs and the target attribute

In [ ]:
features=df.drop('Price',axis=1)
target=df['Price']



features_train,fetures_test,target_train,target_test=train_test_split(features,target,test_size=0.25,random_state=345,shuffle=target)

pre=make_column_transformer((OrdinalEncoder(),[i for i in df.columns  if df[i].dtype !='int']),
                            (StandardScaler(),[i for i in df.columns  if df[i].dtype =='int' and i!='Price']))

pre.fit(features)
fetures_train_encode=pre.transform(features_train)
fetures_test_encode=pre.transform(features_train)

In [ ]:
time_model=[]
RMSE=[]


### Создадим функцию и обучим несколько моделей. / Create study function and leanr some models
* Так же я решил использовать SelectKBest так как у нас достаточно большая выборка а исходя их данных которые у нас были ранее можно ограничить чать признаков для того что бы обучении проходило быстрее
* Изначально я хотел использовать полностью конвеер без функции но так как нам нужно еще учесть и время обучения какждой модели решил разюить все отдельно.

ENG 
* I also decided to use SelectKBest because we have a fairly large sample, and based on the data we had previously, we can limit some features to speed up training.
* Initially, I wanted to use the pipeline completely without a function, but since we also need to take into account the training time of each model, I decided to run everything separately.

In [ ]:
def study(features,target,param,model):
    
    
    
    feature_selection = SelectKBest(score_func=f_regression)

    pipeline = Pipeline([('selection',feature_selection),('model',model)])
    

    grid = RandomizedSearchCV(pipeline, param, cv=5,scoring='neg_root_mean_squared_error')
    grid.fit(features, target)
    
    print('RMSE:', grid.best_score_)
    return grid

In [ ]:


start_time = time.time()
cat=(study(fetures_train_encode,target_train,{'selection__k': [3, 6, 10],'model__learning_rate':[0.01,0.1,1],'model__l2_leaf_reg': [0.1, 1, 10],'model__n_estimators': [100, 500, 100],'model__depth':[4,6,8]},CatBoostRegressor(verbose=False)))
RMSE.append(cat.best_score_)
end_time = time.time()
time_model.append(end_time-start_time)


In [ ]:

start_time = time.time()
forest=(study(fetures_train_encode,target_train,{'selection__k': [3, 6, 10],'model__n_estimators': range(10,70,20),'model__max_depth': [4, 6, 8],'model__min_samples_split': [2, 4, 8]},RandomForestRegressor(random_state=1224)))
RMSE.append(forest.best_score_)
end_time = time.time()
time_model.append(end_time-start_time)

In [ ]:
start_time = time.time()
light=(study(fetures_train_encode,target_train,{'selection__k': [3, 6, 10],'model__learning_rate': [0.01, 0.1, 1],'model__num_leaves': [32, 64, 128],'model__max_depth': [4, 6, 8]},LGBMRegressor()))
RMSE.append(light.best_score_)
end_time = time.time()
time_model.append(end_time-start_time)

In [ ]:
start_time = time.time()
tree=(study(fetures_train_encode,target_train,{'selection__k': [3, 6, 10],'model__max_depth': range(3,10),'model__min_samples_split': [2, 4, 8]},DecisionTreeRegressor()))
RMSE.append(tree.best_score_)
end_time = time.time()
time_model.append(end_time-start_time)

In [ ]:
def predict_time(model,features):
  start_time = time.time()
  model.predict(features)
  end_time = time.time()
  return (end_time-start_time)

## Анализ моделей / Model analysis

In [ ]:
models=pd.DataFrame({'Time':time_model,'RMSE':(RMSE)},index=['CatBoost','RandomForest','LightGBM','DecisionTree'])
models['RMSE']=models['RMSE']*-1
models

In [ ]:
models.plot(kind='bar',grid=True,figsize=(6,6))
pd.Series(train_time,index=['CatBoost','RandomForest','LightGBM','DecisionTree'])

In [ ]:
predict_t=pd.Series([predict_time(i,fetures_test_encode) for i in [cat,forest,light,tree]],index=['CatBoost','RandomForest','LightGBM','DecisionTree'])
predict_t.plot(kind='bar',grid=True,figsize=(5,5))

### Вывод / Conclusion

Мы видим что модель CatBoost показывает лучший показатель в метрике RMSE ,но модель самая долгая ,если искать лучший показать по времени то это определенно Древо решений ,но метрика RMSE одна из самых худших хотя и удовлетворяет условие,но если брать что то среднее между качеством и временем то LightGBMR подходит лучше всего он показывает и относительно хороший показтель метрики и скорость своего обучения.В связи с этим мы остановимся на этой модели,осталось проверить на переобученность .

ENG:

We see that the CatBoost model shows the best performance in the RMSE metric, but it's the slowest model. If we are looking for the best performance in terms of time, then definitely the Decision Tree is the winner, but its RMSE metric is one of the worst, even though it meets the requirement. However, if we're looking for a balance between quality and time, then LightGBM fits the best. It shows relatively good performance in terms of the metric and its training speed. With this in mind, we will settle on this model, the remaining task is to check for overfitting.

In [ ]:
print(f'RMSE модели с обучающими данными : {light.best_score_}')
print(f'RMSE модели с тестовыми данными : {np.sqrt(mean_squared_error(target_test,light.predict(fetures_test)))}')

Мы видим что разницы практически нету в связи с этим окончательно выбираем данную модель / We see that there is practically no difference in this regard, we finally choose this model